<a href="https://colab.research.google.com/github/handeo/SBTi-finance-tool/blob/main/examples/3_what-if-analysis_albaraka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SBTi-Finance Tool - What-if Analysis
In this notebook we show you how to analyse what impact different actions would have on the temperature score of your portfolio.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details on the different what-if scenarios.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular.


## Setting up
First we will set up the imports, data providers, and load the portfolio.

For more examples of this process, please refer to notebook 1 & 2 ([analysis](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) and [quick calculation example](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/2_quick_temp_score_calculation.ipynb)).

In [1]:
%pip install sbti-finance-tool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a

In [2]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [8]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/Data_Provider_norevenue.xlsx"):
    urllib.request.urlretrieve("https://github.com/handeo/SBTi-finance-tool/raw/main/data/Data_Provider_norevenue.xlsx", "data/Data_Provider_norevenue.xlsx")
if not os.path.isfile("data/portfolio_data.csv"):
    urllib.request.urlretrieve("https://github.com/handeo/SBTi-finance-tool/raw/main/data/portfolio_data.csv", "data/portfolio_data.csv")

In [10]:
provider = ExcelProvider(path="data/Data_Provider_norevenue.xlsx")
df_portfolio = pd.read_csv("data/portfolio_data.csv", encoding="iso-8859-1")
df_portfolio.columns = ['company_name'] + list(df_portfolio.columns[1:])
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
portfolio_data = SBTi.utils.get_data([provider], companies)
scenarios = {}

Status code from fetching the CTA file: 200, 200 = OK


## Base scenario
To determine what the effect is of running a certain scenario, we first need a base line. To do so, first we calculate the portfolio temperature score, without defining a scenario.

We can then analyse the effect of the actions in our scenario's by comparing the temperature score medium-term timeframe in the combined s1s2s3 scope. Of course you can use multiple, or different criteria and also change the scope used to calculate the scores.

In [11]:
# Set the time frames and scopes for the scoring          other options displayed below. in this NB we use MID and S1S2S3
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # ETimeFrames: SHORT MID and LONG
scopes = [EScope.S1S2S3]                                # EScopes: S1, S2, S3, S1S2 and S1S2S3

Below we compute the temperature scores for all the timeframe and scope combinations. Subsequently, one can query the score of interest as shown in the last line in the cell below.

In [12]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_score = base_scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Base Scenario': base_score})
print("The portfolio's base scenario temperature score (mid - S1S2S3) is {c:.2f} degrees celcius".format(c=base_score))

The portfolio's base scenario temperature score (mid - S1S2S3) is 3.20 degrees celcius


## Scenario 1
In scenario 1 we engage companies to set emissions reduction targets of at least 2 degrees Celsius.

In practice, in the model, this means that we change the default score (i.e. the score when a company didn't set a target) to 2 degrees Celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_one = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_one)
scenario_1_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 1': scenario_1_score})
print("The portfolio's scenario 1 temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_1_score))

## Scenario 2
In scenario 2 we engage with companies to influence them to set SBTi approved emissions reduction targets. This means that the maximum score for all targets will be capped at 1.75 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_2_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 2': scenario_2_score})
print("The portfolio's scenario 2 temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_2_score))


## Scenario 3a
In scenario 3a we engage the top 10 contributing companies to the portfolio temperature score to influence them to set emission reduction targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_3a_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 3a': scenario_3a_score})
print("The portfolio's scenario 3a temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_3a_score))

## Scenario 3b
In scenario 3b we engage the top 10 contributing companies to the portfolio temperature score to influence them to set SBTi-approved targets. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_3b_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 3b': scenario_3b_score})
print("The portfolio's scenario 3b temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_3b_score))

## Scenario 4a
In scenario 4a we engage with any company in the portfolio to influence them to set emission reduction targets. We define these companies by adding "`TRUE`" in the `engagement_target`-column in the portfolio input-file.

This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees Celsius.

Scenario 4a and 4b gives you a lot of flexibility, as you can define the companies with which you engage freely and for example focus your engagement efforts on certain sectors, countries, market cap buckets to ensure that you spend your time wisely.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4a_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 4a': scenario_4a_score})
print("The portfolio's scenario 4a temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_4a_score))

## Scenario 4b
In scenario 4b we can engage with any company in the portfolio and manage to influence them to set SBTi-approved targets. We define these companies by adding "`TRUE`" in the `engagement_target`-column in the portfolio input-file.

This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4b_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 4b': scenario_4b_score})
print("The portfolio's scenario 4b temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_4b_score))

To see which scenario provides the most suitable outcome we print an overview of all scenarios:

In [ ]:
print('{:<20s}{}'.format('', 'Temp score'))
print('{:<20s}{}'.format('Scenario overview', '(mid - S1S2S3):'))
for scenario, score in scenarios.items():
    print('{:<20s}{:.2f}'.format(scenario, score))